In [1]:
import os
import pandas as pd
import numpy as np
import json

In [1]:
import torch
from minigpt4.models.rec_model import disabled_train
from minigpt4.models.rec_base_models import MatrixFactorization, MF_linear,LightGCN, SASRec, Personlized_Prompt, random_mf, Soft_Prompt, RecEncoder_DIN
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
class Rec_config:
    def __init__(self, user_num, item_num, embedding_size):
        self.user_num = user_num
        self.item_num = item_num
        self.embedding_size = embedding_size

/data/yuqihang/envs/envs/collm/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/data/yuqihang/envs/envs/collm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [2]:
def init_rec_encoder(rec_model, config):
    if rec_model == "MF":
        print("### rec_encoder:", "MF")
        rec_model = MatrixFactorization(config)
    elif rec_model == "lightgcn":
        print("### rec_encoder:", "lightgcn")
        rec_model = LightGCN(config)
    elif rec_model == "sasrec":
        print("### rec_encoder:", "sasrec")
        rec_model = SASRec(config)
    elif rec_model == "DIN":
        print("### rec_encoder:", "DIN")
        rec_model = RecEncoder_DIN(config)
    else:
        rec_model = None
        warnings.warn(" the input rec_model is not MF, LightGCN or sasrec, or DCN, we won't utilize the rec_encoder directly.")
    return rec_model

In [12]:
data_dir = "/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/group_1"
train_ = pd.read_pickle(os.path.join(data_dir,"train_ood2.pkl"))
valid_ = pd.read_pickle(os.path.join(data_dir,"valid_ood2.pkl"))
test_ = pd.read_pickle(os.path.join(data_dir,"test_ood2.pkl"))
reason_ = pd.read_pickle(os.path.join(data_dir,"reason_ood2.pkl"))

In [4]:
data_group_dir = "/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/group"

In [5]:
data_group = os.path.join(data_dir,"user_group.json")
with open(data_group,'r') as f:
    data_group_dict = json.load(f)

In [6]:
type(data_group_dict['0'][0])

int

In [13]:
len(train_),len(valid_),len(test_),len(reason_)

(401399, 34385, 33962, 2225)

In [9]:
len(train_idx),len(valid_idx),len(test_idx),len(reason_idx),len(valid_small_idx)


(249466, 21877, 22300, 2275, 10938)

In [8]:
for idx in data_group_dict:
    data_group_dir_idx = f'{data_group_dir}_{idx}'
    os.makedirs(data_group_dir_idx,exist_ok=True)
    train_idx = train_[train_['uid'].isin(data_group_dict[idx])]
    valid_idx = valid_[valid_['uid'].isin(data_group_dict[idx])]
    # valid_small_idx = valid_small[valid_small['uid'].isin(data_group_dict[idx])]
    valid_small_idx = valid_idx.sample(frac=0.5,random_state=2025)
    test_idx = test_[test_['uid'].isin(data_group_dict[idx])]
    reason_idx = reason_[reason_['uid'].isin(data_group_dict[idx])]
    train_idx.to_pickle(os.path.join(data_group_dir_idx,"train_ood2.pkl"))
    valid_idx.to_pickle(os.path.join(data_group_dir_idx,"valid_ood2.pkl"))
    valid_small_idx.to_pickle(os.path.join(data_group_dir_idx,"valid_small_ood2.pkl"))
    test_idx.to_pickle(os.path.join(data_group_dir_idx,"test_ood2.pkl"))
    reason_idx.to_pickle(os.path.join(data_group_dir_idx,"reason_ood2.pkl"))


In [4]:
print('Loading Rec_model')
rec_model = "MF"
rec_config = Rec_config(user_num=int(user_num), item_num=int(item_num), embedding_size=256)
rec_encoder = init_rec_encoder(rec_model, rec_config)

pretrained_rec = "/data/yuqihang/result/CoLLM/checkpoints/mf/0228_booknew_best_model_d256_lr0.001_wd1e-06.pth"
rec_encoder.load_state_dict(torch.load(pretrained_rec, map_location="cpu"))
print("successfully load the pretrained model......")
for name, param in rec_encoder.named_parameters():
    param.requires_grad = False
rec_encoder = rec_encoder.eval()
rec_encoder.train = disabled_train
print("freeze rec encoder")
print('Loading Rec_model Done')

Loading Rec_model
### rec_encoder: MF
creat MF model, user num: 22686 item num: 47059
successfully load the pretrained model......
freeze rec encoder
Loading Rec_model Done


/tmp/ipykernel_2994867/1765465668.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rec_encoder.load_state_dict(torch.load(pretrained_rec, map_location="cpu"))


In [5]:
users = list(train_.uid)
users.extend(list(test_.uid))
users.extend(list(valid_.uid))
users = list(set(users))

In [6]:
user_embeds = rec_encoder.user_encoder(torch.tensor(users)).detach().numpy()

In [9]:
# 可视化 (使用t-SNE降维到2D)
def visualize_clusters(embeddings, labels):
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter)
    plt.title('t-SNE Visualization of User Clusters')
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')
    plt.show()

In [10]:
user_ids = users
embeddings = user_embeds
# 数据标准化 (对基于距离的聚类方法很重要)
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)
# 执行聚类 (这里以K-Means为例)
cluster_labels = kmeans_clustering(embeddings_scaled, max_clusters=10)
# 将结果保存到DataFrame
results = pd.DataFrame({
    'user_id': user_ids,
    'cluster': cluster_labels
})
visualize_clusters(embeddings_scaled, cluster_labels)
# 查看聚类结果
print("聚类结果统计:")
print(results['cluster'].value_counts())
# 保存结果到CSV
# results.to_csv('user_clusters.csv', index=False)

/data/yuqihang/envs/envs/collm/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


: 